In [2]:
import gradio as gr
import mmcv
import os
from utils import list_from_folder, inference_plot, get_detector

In [13]:
# 1. get all the configs from the config directory
config_prefix = './configs'
models = list_from_folder(config_prefix, mode='d', full_path=True)
configs = {model.split('\\')[-1] : 
           [i for i in list_from_folder(model, mode='f', full_path=True) if i.endswith('.py')]
           for model in models}

# 2. mmcv.Config.fromfile to transfer the configs file
for i, j in configs.items():
    print(i, len(j))

baselines 8
cfa 3
convnext 2
csl 1
dcfl 7
gliding_vertex 1
gwd 3
g_reppoints 1
kfiou 15
kld 11
model_customized 8
oriented_rcnn 5
oriented_reppoints 3
r3det 4
redet 6
roi_trans 7
rotated_atss 4
rotated_faster_rcnn 1
rotated_fcos 4
rotated_reppoints 4
rotated_retinanet 13
s2anet 4
sasm_reppoints 1
_base_ 3


In [23]:
# get the config files and transfer 
conf_oriented_rcnn = configs['oriented_rcnn']

# conf_oriented_rcnn = [mmcv.Config.fromfile(i) for i in conf_oriented_rcnn]
model_dicts_oriented_rcnn = {os.path.split(i)[-1].strip('.py') : mmcv.Config.fromfile(i) for i in conf_oriented_rcnn}
for model, model_dict in model_dicts_oriented_rcnn.items():
    print(model, model_dict)

# get the dict and dynamically gen config configuration file
model_dict = model_dicts_oriented_rcnn['oriented_rcnn_r50_fpn_1x_dota_le90']

oriented_rcnn_r50_fpn_1x_dota_le90 Config (path: ./configs\oriented_rcnn\oriented_rcnn_r50_fpn_1x_dota_le90.py): {'dataset_type': 'DOTADataset', 'data_root': 'data/split_ss_dota/', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'train_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True}, {'type': 'RResize', 'img_scale': (1024, 1024)}, {'type': 'RRandomFlip', 'flip_ratio': [0.25, 0.25, 0.25], 'direction': ['horizontal', 'vertical', 'diagonal'], 'version': 'le90'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}], 'test_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1024, 1024), 'flip': False, 'transforms': [{'type': 'RResize'}, {'type': 'Normalize', 'mean': [123.675, 116

In [33]:
# dump the file into a config file
model_dict.dump('./config_test.py')

# let's just think about how to make an abstract interface for changing the config
for i, j in model_dict.items():
    print(i, j)

dataset_type DOTADataset
data_root data/split_ss_dota/
img_norm_cfg {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}
train_pipeline [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True}, {'type': 'RResize', 'img_scale': (1024, 1024)}, {'type': 'RRandomFlip', 'flip_ratio': [0.25, 0.25, 0.25], 'direction': ['horizontal', 'vertical', 'diagonal'], 'version': 'le90'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}]
test_pipeline [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1024, 1024), 'flip': False, 'transforms': [{'type': 'RResize'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'ty

In [41]:
# the solution to dynamically rendering the components
"""
    1. the main block should have a list of all the blocks
    2. re render components by redifining the children of the blocks
    3. the new task  
        1. customize a new component meant for the dict configuration
        2. test the re-rendering of the components
"""
max_textboxes = 10


def variable_outputs(k):
    k = int(k)
    return [gr.Textbox(visible=True)]*k + [gr.Textbox(visible=False)]*(max_textboxes-k)

with gr.Blocks() as demo:
    s = gr.Slider(1, max_textboxes, value=max_textboxes, step=1, label="How many textboxes to show:")
    textboxes = []
    for i in range(max_textboxes):
        t = gr.Textbox(f"Textbox {i}")
        textboxes.append(t)

    s.change(variable_outputs, s, textboxes)

demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
